November 3 , 2023

Running this example on my brand spanking new 4090!

[How to Load LLama 13b for Inference on a single RTX 4090](https://codehammer.io/how-to-load-llama-13b-for-inference-on-a-single-rtx-4090/)

docker container start hfpt_Oct28

( docker container id is c9b676310ea0 )

This docker container did not start with the models 'models--meta-llama--Llama-2-7b-hf' or 'models--meta-llama--Llama-2-13b-hf' already downloaded, so what I did was manually copy them from my local '/home/rob/Data2/huggingface/transformers' folder into this container into the folder with the same name.

docker cp /home/rob/Data2/huggingface/transformers/models--meta-llama--Llama-2-7b-hf c9b676310ea0://home/rob/Data2/huggingface/transformers

docker cp /home/rob/Data2/huggingface/transformers/models--meta-llama--Llama-2-13b-hf c9b676310ea0://home/rob/Data2/huggingface/transformers

In [5]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

In [5]:
!ls /home/rob/Data2/huggingface/transformers

models--bert-base-cased
models--bigscience--bloomz-560m
models--decapoda-research--llama-13b-hf
models--facebook--bart-large-cnn
models--google--pegasus-cnn_dailymail
models--google--vit-base-patch16-224-in21k
models--gpt2
models--gpt2-xl
models--meta-llama--Llama-2-13b-hf
models--meta-llama--Llama-2-7b-hf
models--mistralai--Mistral-7B-Instruct-v0.1
models--mistralai--Mistral-7B-v0.1
models--nvidia--mit-b0
models--openai-gpt
models--roberta-large
models--robkayinto--vit-base-patch16-224-in21k-finetuned-lora-food101
models--t5-large
models--transformersbook--pegasus-samsum
models--xlm-roberta-base
tmp3i5avt2i
tmp560c_s3e
tmp8is86yg8
tmpea0zbrrj
tmplm91b70r
tmpni3ccozw
version.txt


Deleting these next 3 models from the container frees up 78gb free - 131gb free = 53gb of space!

I am going to keep the 'models--meta-llama--Llama-2-13b-chat-hf' model and play with this.

In [3]:
# !rm -rf /home/rob/Data2/huggingface/transformers/models--meta-llama--Llama-2-7b-hf
# docker cp /home/rob/Data2/huggingface/transformers/models--meta-llama--Llama-2-7b-hf c9b676310ea0://home/rob/Data2/huggingface/transformers
# Successfully copied 13.5GB to c9b676310ea0://home/rob/Data2/huggingface/transformers


In [3]:
# !rm -rf /home/rob/Data2/huggingface/transformers/models--mistralai--Mistral-7B-Instruct-v0.1

In [4]:
# !rm -rf /home/rob/Data2/huggingface/transformers/models--meta-llama--Llama-2-7b-chat-hf
# docker cp /home/rob/Data2/huggingface/transformers/models--meta-llama--Llama-2-7b-chat-hf c9b676310ea0://home/rob/Data2/huggingface/transformers

In [11]:
# !rm -rf /home/rob/Data2/huggingface/transformers/models--meta-llama--Llama-2-13b-hf
# docker cp /home/rob/Data2/huggingface/transformers/models--meta-llama--Llama-2-13b-hf c9b676310ea0://home/rob/Data2/huggingface/transformers
# Successfully copied 26GB to c9b676310ea0://home/rob/Data2/huggingface/transformers


In [6]:
# !rm -rf /home/rob/Data2/huggingface/transformers/models--meta-llama--Llama-2-13b-chat-hf
# docker cp /home/rob/Data2/huggingface/transformers/models--meta-llama--Llama-2-13b-chat-hf c9b676310ea0://home/rob/Data2/huggingface/transformers

All of these models load to the 4090 

In [7]:
checkpoint = 'meta-llama/Llama-2-7b'    # Nope!
checkpoint = 'meta-llama/Llama-2-7b-hf' # Works! 7718MiB

checkpoint = 'meta-llama/Llama-2-7b-chat'    #Nope! 
checkpoint = 'meta-llama/Llama-2-7b-chat-hf' # Works! 7718MiB

checkpoint = 'meta-llama/Llama-2-13b'     # Nope!
checkpoint = 'meta-llama/Llama-2-13b-hf'  # Works! 14006MiB

checkpoint = 'meta-llama/Llama-2-13b-chat' # Nope!
checkpoint = 'meta-llama/Llama-2-13b-chat-hf' #Works! 14006MiB


In [2]:
# Does this work for mistralai/Mistral-7B-v0.1 ?? Yup! It's downloading! Nice!
# docker cp c9b676310ea0://home/rob/Data2/huggingface/transformers/models--mistralai--Mistral-7B-v0.1 /home/rob/Data2/huggingface/transformers/
checkpoint = 'mistralai/Mistral-7B-v0.1'
model = AutoModelForCausalLM.from_pretrained(checkpoint, load_in_8bit=True) # 8236 VRAM

# model = AutoModelForCausalLM.from_pretrained(checkpoint)

# 207m 33.2s

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [6]:
# OK! This now works! Sweet! 7660MiB
checkpoint = 'meta-llama/Llama-2-7b-hf'
model = AutoModelForCausalLM.from_pretrained(checkpoint, load_in_8bit=True)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [9]:
# This works! and is already part of this container! 7660MiB
checkpoint = 'meta-llama/Llama-2-7b-chat-hf'
# model = AutoModelForCausalLM.from_pretrained(checkpoint, load_in_8bit=True)

In [10]:
# OK! This now works! Sweet! 13948MiB
checkpoint = 'meta-llama/Llama-2-13b-hf'
# model = AutoModelForCausalLM.from_pretrained(checkpoint, load_in_8bit=True)

In [11]:
%%time
# This works! and is already part of this container! 13948MiB
checkpoint = 'meta-llama/Llama-2-13b-chat-hf'
model = AutoModelForCausalLM.from_pretrained(checkpoint, load_in_8bit=True)

# Loading if from the new docker location found at /home/rob/Data3/docker-data 
# CPU times: user 5.84 s, sys: 10.4 s, total: 16.2 s
# Wall time: 1min 10s

# Load time if we use the default /var/lib/docker folder ... way faster!!
# CPU times: user 6.16 s, sys: 8.44 s, total: 14.6 s
# Wall time: 16.3 s

# and retested a 2nd time, just to test if using /home/rob/Data3/docker-data
# does indeed make things slower, and YES IT DOES!!
# CPU times: user 6.07 s, sys: 8.88 s, total: 15 s
# Wall time: 1min

# Excellent! Moved the docker location to /home/rob/Data3/docker-data ... and now it's fast again! Nice!
# CPU times: user 5.11 s, sys: 7.26 s, total: 12.4 s
# Wall time: 14.9 s

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

CPU times: user 5.11 s, sys: 7.26 s, total: 12.4 s
Wall time: 14.9 s


In [7]:
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [8]:
pipe = pipeline(
            task="text-generation",
            model=model,
            tokenizer=tokenizer,
            max_new_tokens=64,
            do_sample=True,
            temperature=0.7,
        )

In [9]:
%%time
print(pipe("Question: What is the difference between a car and a truck?\nAnswer: "))

[{'generated_text': 'Question: What is the difference between a car and a truck?\nAnswer: 18-wheelers.\nThis was the last one, and I was done.\nI’d like to thank everyone who participated in the contest, especially those who tried their hand at writing. I was amazed at the creativity and talent that was displayed, and I hope that everyone learned something about'}]
CPU times: user 8.86 s, sys: 485 ms, total: 9.34 s
Wall time: 9.57 s


In [6]:
pipe = pipeline(
            task="text-generation",
            model=model,
            tokenizer=tokenizer,
            max_new_tokens=256,
            do_sample=True,
            temperature=0.7,
        )

In [7]:
%%time
print(pipe("Question: What is the difference between a car and a truck?\nAnswer: "))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


[{'generated_text': 'Question: What is the difference between a car and a truck?\nAnswer: 20 mpg!\n\n## 5.\n\nQuestion: What can you do to a car that’s never been made?\nAnswer: Win a race!\n\n## 6.\n\nQuestion: Can you name a car that has an air conditioner?\nAnswer: An F1!\n\n## 7.\n\nQuestion: How can you tell a car to go faster?\nAnswer: Press the accelerator pedal!\n\n## 8.\n\nQuestion: What do you call a car that eats people?\nAnswer: A cannibal!\n\n## 9.\n\nQuestion: Can you name a car that has no engine?\nAnswer: A taxi!\n\n## 10.\n\nQuestion: Why did the man cross the road?\nAnswer: Because it was a chicken!\n\n## 11.\n\nQuestion: What do you call a car with no doors?\nAnswer: A car!\n\n## 12.\n\nQuestion: What do you call a car with no doors?\nAnswer: A car!\n\n## 13.\n\nQuestion: What do you call'}]
CPU times: user 40.9 s, sys: 181 ms, total: 41.1 s
Wall time: 41.1 s


In [17]:
# while True:
#     question = input("Question: ")
#     prompt = f"Question: {question}\nAnswer: "
#     result = pipe(prompt)[0]["generated_text"][len(prompt):]
#     print(f"Answer: {result}")

In [18]:
%%time
print(pipe("Question: What is the square root of 25?\nAnswer: "))

[{'generated_text': 'Question: What is the square root of 25?\nAnswer: √25 = 5'}]
CPU times: user 1.14 s, sys: 3.43 ms, total: 1.15 s
Wall time: 1.15 s
